In [ ]:
import pandas as pd
import numpy as np
import utils.openfilegui
import common.reading
import common.keys as keys
import common.analysis as al
import matplotlib.pyplot as plt

In [ ]:
file = utils.openfilegui.gui_fname("./")

In [ ]:
accountsFile = utils.openfilegui.gui_fname("./")

In [ ]:
iadf = pd.read_csv(accountsFile)
iadf

In [ ]:
df = common.reading.get_df(file)

In [ ]:
expense_ = df[df[keys.TYPE] == keys.EXPENSE]
income_ = df[df[keys.TYPE] == keys.INCOME]
invest_ = df[(df[keys.TYPE] == keys.TRANSFER) & (
    (df[keys.CATEGORY] == keys.INVESTING)) | (df[keys.CATEGORY] == keys.DEINVESTING)]

In [ ]:
expenseM = al.transformByCat(expense_, 'M', keys.CATEGORY)
expenseM_PC = al.transformByCat(expense_, 'M', keys.PARENT_CATEGORY)
incomeM = al.transformByCat(income_, 'M', keys.CATEGORY)
incomeM_PC = al.transformByCat(income_, 'M', keys.PARENT_CATEGORY)
investM = al.transformByCat(invest_, 'M', keys.ACCOUNT_RECEIVABLE)
totalM = al.merge(expenseM, incomeM, investM)

expenseY = al.transformByCat(expense_, 'Y', keys.CATEGORY)
expenseY_PC = al.transformByCat(expense_, 'Y', keys.PARENT_CATEGORY)
incomeY = al.transformByCat(income_, 'Y', keys.CATEGORY)
incomeY_PC = al.transformByCat(income_, 'Y', keys.PARENT_CATEGORY)
investY = al.transformByCat(invest_, 'Y', keys.ACCOUNT_RECEIVABLE)
totalY = al.merge(expenseY, incomeY, investY)

In [ ]:
def year_report(year) -> str:
    e = al.get_most_changed(expenseY, year).to_markdown()
    epc = al.get_most_changed(expenseY_PC, year).to_markdown()
    i = al.get_most_changed(incomeY, year).to_markdown()
    ipc = al.get_most_changed(incomeY_PC, year).to_markdown()

    res = f"### Expense changing\n{e}\n"
    res += f"### Expense category changing\n{epc}\n"
    res += f"### Income changing\n{i}\n"
    res += f"### Income category changing\n{ipc}\n"

    return res

In [ ]:
year = 2023
from pathlib import Path
rep = year_report(year)
directory = Path(file).parent
rep_file = f"{str(directory)}/report_{year}.md"
with open(rep_file, "w") as f:
    f.write(rep)

In [ ]:
def makePies(data: pd.DataFrame):
    years = len(data.T.columns)
    # fig, axes = plt.subplots(nrows=len(data.T.columns), figsize=(30,30))
    for year_num in range(years):
        year = data.T.columns[year_num]
        df = al.getDFForPie(data, year, 2)
        if df is None:
            continue
        # df.plot(ax=axes[year_num], kind='pie', autopct='%i%%', y=year, legend=False, subplots=True)
        plt.figure(figsize=(40, 40))
        df.plot(kind='pie', autopct='%i%%',
                y=year, legend=False, subplots=True)


def make_single_lines(data: pd.DataFrame):
    data = data.loc[:, data.columns != keys.TOTAL]
    for c in data.columns:
        df = data[c]

        plt.figure(figsize=(6, 6))
        plt.title(c)
        plt.grid(True)
        df.plot(subplots=True)

In [ ]:
make_single_lines(incomeY_PC)

In [ ]:
make_single_lines(expenseY_PC)

In [ ]:
expenseM_PC.loc[:, expenseM_PC.columns != keys.TOTAL].plot()
plt.grid(True)

In [ ]:
incomeM_PC.loc[:, incomeM_PC.columns != keys.TOTAL].plot()
plt.grid(True)

In [ ]:
totalM.loc[:, totalM.columns != keys.TOTAL].plot()
plt.grid(True)

In [ ]:
ax = totalY.iloc[:, totalY.columns != keys.TOTAL].plot(kind='bar')
totalY[keys.EXPENSE_SHARE].plot(ax=ax, secondary_y=True, color='r', ylim=(0,100))
totalY[keys.INVESTING_SHARE].plot(ax=ax, secondary_y=True, color='y', ylim=(0,100))
plt.grid(True)

In [ ]:
makePies(expenseY)

In [ ]:
makePies(incomeY)

In [ ]:
makePies(investY)

In [ ]:
expenseY_PC.iloc[:, expenseY_PC.columns != keys.TOTAL].plot(kind='barh', stacked=True)
plt.grid(True)

In [ ]:
incomeY_PC.iloc[:, incomeY_PC.columns != keys.TOTAL].plot(kind='barh', stacked=True)
plt.grid(True)

In [ ]:
totalM.loc[:, totalM.columns == keys.TOTAL].plot()
plt.grid(True)

In [ ]:
total_income = totalY[keys.INCOME].sum()
total_expense = totalY[keys.EXPENSE].sum()
total_invest = totalY[keys.INVESTING].sum()

print("Total: {:,}".format(total_income - total_expense))
print("Free: {:,}".format(total_income - total_expense - total_invest))
print("Invested: {:,}".format(total_invest))


In [ ]:
df = al.get_df_mapped(investM, iadf)
df.plot(kind='area')

In [ ]:
ndf = df.div(df.sum(axis=1), axis=0)
ndf.plot(kind='area')